## Initialization code

In [1]:
%matplotlib inline

In [2]:
import os
import tempfile
import pandas as pd
import numpy as np
import networkx as nx

## Compilation and test of waf command

The file iterative_avg_consensus.cc must be copied in the scratch folder of ns3

In [3]:
!../cmake-build-debug/iterative_avg_consensus --PrintHelp

iterative_avg_consensus [Program Arguments] [General Arguments]

Program Arguments:
    --nNodes:            Number of nodes in the simulation [2]
    --secsToRun:         number of seconds to simulate [10]
    --makhoul:           use the example of makhoul thesis [0]
    --path:              output path [/tmp]
    --self_stab:         self stabilization mode [0]
    --positionRng:       define the random generator for the position allocator [ns3::UniformRandomVariable[Min=0.0|Max=100.0]]
    --distance:          define the maximum distance for connectivity [30]
    --async:             set async mode for diffusion algorithm [0]
    --seed:              set seed value [42]
    --run_id:            set run id [0]
    --graph_correction:  run the graph correction for non-connected components [1]

General Arguments:
    --PrintGlobals:              Print the list of globals.
    --PrintGroups:               Print the list of groups.
    --PrintGroup=[group]:        Print all TypeIds of g

## Simulation meta command

NS_LOG=IterativeAvgConsensus=level_info 

In [4]:
def start_simulation(nnodes=20, distance=30, secs=50, test=0, self_stab=0, async=0, run_id=0,
                    graph_correction=1):
    if test > 0:
        nnodes = 4
    path = tempfile.mkdtemp()
    cmd = '''../cmake-build-debug/iterative_avg_consensus \
--nNodes={nnodes} --distance={distance} \
--secsToRun={secs} --makhoul={test} --path={path} --self_stab={self_stab} --async={async} \
--run_id={run_id} --graph_correction={graph_correction}  2>&1 >/dev/null \
| grep "^TR" > {path}/ts.tsv'''.format(nnodes=nnodes,
                                                                            distance=distance,
                                                                           secs=secs,
                                                                           path=path,
                                                                           test=test,
                                                                           self_stab=self_stab,
                                                                           async=async,
                                                                           run_id=run_id,
                                                                           graph_correction=graph_correction)
    print cmd
    os.system(cmd)
    trace_df = pd.read_csv("{path}/ts.tsv".format(path=path), sep='\t', 
                       names=["tag", "node", "sim_time", "nit", "nmsg", "z", "x", "energy"])

    G = nx.Graph()    
    positions = {}
    
    with open("{path}/graph.tsv".format(path=path), "r") as fp:
        # Get positions
        for i in range(nnodes):
            nid, x, y = fp.readline().replace("\n", "").split("\t")
            G.add_node("node%s" %nid, pos=(x, y))
            positions["node%s" %nid] = (float(x), float(y))
        # Get connections
        for connection in fp:
            fn, tn = connection.replace("\n", "").split("\t")
            G.add_edge("node%s" %fn, "node%s" %tn)
        G.positions = positions
    return trace_df, G

In [5]:
def get_epsilon(df, col, sim_avg=1.0):
    df["eps"] = abs(df.x - sim_avg)
    return df[[col, "eps"]].groupby(col).agg(np.max)

In [6]:
def get_epsilon_min(df, col, thr=10e-4, sim_avg=1.0):
    pos, _ = np.where(get_epsilon(df, col, sim_avg)<thr)
    if len(pos) > 0:
        return pos[0]
    else:
        return None

## Plotting commands

In [7]:
def compute_convergence(df, epsilon=0.1, sim_avg=1.0, plot=False):
    # 50 ms bucket    
    # Conv vs time
    conv_df = df.copy()
    conv_df["sim_step"] = (conv_df.sim_time/50).astype(int)
    outdf = conv_df[["sim_step", "node", "x"]].groupby(["sim_step", "node"]).agg(np.mean).reset_index()
    eps_min = get_epsilon_min(outdf, col="sim_step", thr=epsilon, sim_avg=sim_avg)
#    if v is not None:
#        outdf = outdf[outdf.sim_step<=v]
    outdf.sim_step *= 50
    conv_df.sim_step *= 50
    pv_df = outdf.pivot(index="sim_step", columns="node", values="x")
    pv_df["nmsg"] = conv_df[["nmsg", "sim_step"]].groupby("sim_step").agg(np.sum)
    
    if plot:
        pv_df.plot(secondary_y="nmsg", title="Convergence vs time", grid=True)
    
    metrics_df = pv_df[["nmsg"]].copy()
    
    pv_df.drop(["nmsg"], axis=1, inplace=True)
    pv_df["energy"] = conv_df[["energy", "sim_step"]].groupby("sim_step").agg(np.mean)
    if plot:    
        pv_df.plot(secondary_y="energy", title="Convergence vs time / energy", grid=True)
    metrics_df["energy"] = pv_df["energy"]
    
    delta_df = outdf.copy()
    delta_df["delta"] = abs(delta_df.x - sim_avg)
    delta_plt = delta_df[["delta", "sim_step"]].groupby("sim_step").agg(np.max)
    if plot:    
        delta_plt.plot(title="Maximum delta vs time", grid=True)
    metrics_df["delta"] = delta_plt["delta"]
    
    return metrics_df, eps_min

    
    '''
    outdf = df.pivot(index="nit", columns="node", values="x")
    outdf["nmsg"] = df[["nmsg", "nit"]].groupby("nit").agg(np.sum)
    if v is not None:
        outdf = outdf[outdf.index<=v]
    outdf.plot(secondary_y="nmsg", title="Convergence vs iterations", grid=True)
    '''
    
    flt_out = outdf.drop(["nmsg"], axis=1)
    flt_out["min_max_delta"] = df[["x", "sim_step"]].groupby("sim_step").agg(np.max) - df[["x", "sim_step"]].groupby("sim_step").agg(np.min)
    pd.DataFrame(index=outdf.nmsg, data=flt_out.values, 
                 columns=flt_out.columns).plot(secondary_y="min_max_delta", 
                                               title="Convergence vs # messages exchanged",
                                              legend='reverse', grid=True)

In [8]:
def plot_graph(G):
    nx.draw(G, G.positions, with_labels = True)

In [9]:
def plot_minmax(df):
    outdf = df.pivot(index="nit", columns="node", values="x")
    outdf["delta"] = df[["x", "nit"]].groupby("nit").agg(np.max) - df[["x", "nit"]].groupby("nit").agg(np.min)
    outdf["delta"].plot()

# Experiments

In [10]:
import os

In [12]:
def experiment(meta_args, epsilon=10e-3, sim_avg=1, nbruns=1, force_run_id=None, xp_name="data"):
    try:
        os.mkdir(xp_name)
    except:
        pass
    if force_run_id is not None:
        nbruns = 1
    for run_id in range(nbruns):
        if force_run_id is not None:
            run_id = force_run_id
        # diffusion
        trace_results, G = start_simulation(run_id=run_id, **meta_args)
        degree_diff = G.degree(G.nodes())
        metrics_diffusion, eps_min_diff = compute_convergence(trace_results, epsilon=epsilon, sim_avg=sim_avg)
        # self-tab
        trace_results, G = start_simulation(run_id=run_id, self_stab=1, **meta_args)
        degree_sstab = G.degree(G.nodes())        
        metrics_sstab, eps_min_sstab = compute_convergence(trace_results, epsilon=epsilon, sim_avg=sim_avg)

        mg = pd.merge(metrics_diffusion, metrics_sstab, how='outer', 
                      left_index=True, right_index=True, suffixes=('_diff', '_sstab'))
        mg["run_id"] = run_id
        mg["xp_name"] = xp_name
        mg["eps_min_diff"] = eps_min_diff
        mg["eps_min_sstab"] = eps_min_sstab       
        mg["degree_diff"] = np.mean(degree_diff.values())
        mg["degree_sstab"] = np.mean(degree_sstab.values())
        mg.to_csv("%s/run_%d.csv" %(xp_name, run_id), sep='\t')
    return mg

## Testing with a 10 nodes and low connectivity example

In [16]:
%%time
EPSILON=10e-3
SIM_AVG=1.
meta_args = dict(nnodes=10, distance=10, secs=15, graph_correction=1)
mg = experiment(meta_args, epsilon=EPSILON, sim_avg=SIM_AVG, nbruns=100, xp_name="nnodes_10-distance_10")

../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=10 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpGN0zj0 --self_stab=0 --async=0 --run_id=0 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpGN0zj0/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=10 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmp6Swx9G --self_stab=1 --async=0 --run_id=0 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmp6Swx9G/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=10 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpT1KZiN --self_stab=0 --async=0 --run_id=1 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpT1KZiN/ts.tsv
../cmake-build-debug/iterative_avg_consensus -

../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=10 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpMgiZBr --self_stab=0 --async=0 --run_id=13 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpMgiZBr/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=10 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmp33dXLZ --self_stab=1 --async=0 --run_id=13 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmp33dXLZ/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=10 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpsnKCNv --self_stab=0 --async=0 --run_id=14 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpsnKCNv/ts.tsv
../cmake-build-debug/iterative_avg_consensu

../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=10 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpuDbsGp --self_stab=0 --async=0 --run_id=26 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpuDbsGp/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=10 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpJTckTF --self_stab=1 --async=0 --run_id=26 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpJTckTF/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=10 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpcw7Ocg --self_stab=0 --async=0 --run_id=27 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpcw7Ocg/ts.tsv
../cmake-build-debug/iterative_avg_consensu

../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=10 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpwiZ6xL --self_stab=0 --async=0 --run_id=39 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpwiZ6xL/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=10 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpW5O1Ek --self_stab=1 --async=0 --run_id=39 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpW5O1Ek/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=10 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpmdryGn --self_stab=0 --async=0 --run_id=40 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpmdryGn/ts.tsv
../cmake-build-debug/iterative_avg_consensu

../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=10 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpMRzf90 --self_stab=0 --async=0 --run_id=52 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpMRzf90/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=10 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpxVEMg0 --self_stab=1 --async=0 --run_id=52 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpxVEMg0/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=10 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpBriDC2 --self_stab=0 --async=0 --run_id=53 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpBriDC2/ts.tsv
../cmake-build-debug/iterative_avg_consensu

../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=10 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpElBjXo --self_stab=0 --async=0 --run_id=65 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpElBjXo/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=10 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpCEvZDh --self_stab=1 --async=0 --run_id=65 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpCEvZDh/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=10 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpixBRS4 --self_stab=0 --async=0 --run_id=66 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpixBRS4/ts.tsv
../cmake-build-debug/iterative_avg_consensu

../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=10 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpsmkH7a --self_stab=0 --async=0 --run_id=78 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpsmkH7a/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=10 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpiRHMO_ --self_stab=1 --async=0 --run_id=78 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpiRHMO_/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=10 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpaWHVmA --self_stab=0 --async=0 --run_id=79 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpaWHVmA/ts.tsv
../cmake-build-debug/iterative_avg_consensu

../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=10 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmppLE3PD --self_stab=0 --async=0 --run_id=91 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmppLE3PD/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=10 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpm8ojUq --self_stab=1 --async=0 --run_id=91 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpm8ojUq/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=10 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpJRs3_7 --self_stab=0 --async=0 --run_id=92 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpJRs3_7/ts.tsv
../cmake-build-debug/iterative_avg_consensu

In [12]:
%%time
EPSILON=10e-3
SIM_AVG=1.
meta_args = dict(nnodes=10, distance=20, secs=15, graph_correction=1)
mg = experiment(meta_args, epsilon=EPSILON, sim_avg=SIM_AVG, nbruns=100, xp_name="nnodes_10-distance_20")

../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=20 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpLZ_zLX --self_stab=0 --async=0 --run_id=0 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpLZ_zLX/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=20 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpVHwKlB --self_stab=1 --async=0 --run_id=0 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpVHwKlB/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=20 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpiT3DP4 --self_stab=0 --async=0 --run_id=1 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpiT3DP4/ts.tsv
../cmake-build-debug/iterative_avg_consensus -

../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=20 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmp0pO1S2 --self_stab=0 --async=0 --run_id=13 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmp0pO1S2/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=20 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpCoCYxW --self_stab=1 --async=0 --run_id=13 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpCoCYxW/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=20 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpegaHLX --self_stab=0 --async=0 --run_id=14 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpegaHLX/ts.tsv
../cmake-build-debug/iterative_avg_consensu

../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=20 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpoXJzV7 --self_stab=0 --async=0 --run_id=26 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpoXJzV7/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=20 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpUgo_Po --self_stab=1 --async=0 --run_id=26 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpUgo_Po/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=20 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpHdvULK --self_stab=0 --async=0 --run_id=27 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpHdvULK/ts.tsv
../cmake-build-debug/iterative_avg_consensu

../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=20 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmphYQoXW --self_stab=0 --async=0 --run_id=39 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmphYQoXW/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=20 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpnpiLW2 --self_stab=1 --async=0 --run_id=39 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpnpiLW2/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=20 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmp6uhtl7 --self_stab=0 --async=0 --run_id=40 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmp6uhtl7/ts.tsv
../cmake-build-debug/iterative_avg_consensu

../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=20 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpQ4LvfP --self_stab=0 --async=0 --run_id=52 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpQ4LvfP/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=20 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmp0heYU3 --self_stab=1 --async=0 --run_id=52 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmp0heYU3/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=20 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpUkpe11 --self_stab=0 --async=0 --run_id=53 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpUkpe11/ts.tsv
../cmake-build-debug/iterative_avg_consensu

../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=20 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpOl_nQe --self_stab=0 --async=0 --run_id=65 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpOl_nQe/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=20 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpn1LNEz --self_stab=1 --async=0 --run_id=65 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpn1LNEz/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=20 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmp3T50Xi --self_stab=0 --async=0 --run_id=66 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmp3T50Xi/ts.tsv
../cmake-build-debug/iterative_avg_consensu

../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=20 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmp1AA1et --self_stab=0 --async=0 --run_id=78 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmp1AA1et/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=20 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpaIR55O --self_stab=1 --async=0 --run_id=78 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpaIR55O/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=20 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpzP_Sef --self_stab=0 --async=0 --run_id=79 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpzP_Sef/ts.tsv
../cmake-build-debug/iterative_avg_consensu

../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=20 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpOjhsEq --self_stab=0 --async=0 --run_id=91 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpOjhsEq/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=20 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpyjtcMU --self_stab=1 --async=0 --run_id=91 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpyjtcMU/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=20 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpJ2_rLk --self_stab=0 --async=0 --run_id=92 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpJ2_rLk/ts.tsv
../cmake-build-debug/iterative_avg_consensu

In [13]:
%%time
EPSILON=10e-3
SIM_AVG=1.
meta_args = dict(nnodes=10, distance=30, secs=15, graph_correction=1)
mg = experiment(meta_args, epsilon=EPSILON, sim_avg=SIM_AVG, nbruns=100, xp_name="nnodes_10-distance_30")

../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=30 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpvalYtj --self_stab=0 --async=0 --run_id=0 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpvalYtj/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=30 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpnEl0Fz --self_stab=1 --async=0 --run_id=0 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpnEl0Fz/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=30 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpQlK0iz --self_stab=0 --async=0 --run_id=1 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpQlK0iz/ts.tsv
../cmake-build-debug/iterative_avg_consensus -

../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=30 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmp7Q7tDH --self_stab=0 --async=0 --run_id=13 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmp7Q7tDH/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=30 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpIihZMW --self_stab=1 --async=0 --run_id=13 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpIihZMW/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=30 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpRBBlGR --self_stab=0 --async=0 --run_id=14 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpRBBlGR/ts.tsv
../cmake-build-debug/iterative_avg_consensu

../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=30 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpHDBWWN --self_stab=0 --async=0 --run_id=26 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpHDBWWN/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=30 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpbkXmHr --self_stab=1 --async=0 --run_id=26 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpbkXmHr/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=30 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmp16MpDw --self_stab=0 --async=0 --run_id=27 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmp16MpDw/ts.tsv
../cmake-build-debug/iterative_avg_consensu

../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=30 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpfFFup3 --self_stab=0 --async=0 --run_id=39 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpfFFup3/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=30 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpW9uB2D --self_stab=1 --async=0 --run_id=39 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpW9uB2D/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=30 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpV2kGDc --self_stab=0 --async=0 --run_id=40 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpV2kGDc/ts.tsv
../cmake-build-debug/iterative_avg_consensu

../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=30 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpxb7dx4 --self_stab=0 --async=0 --run_id=52 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpxb7dx4/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=30 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmp_iiPTd --self_stab=1 --async=0 --run_id=52 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmp_iiPTd/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=30 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmp62ZXqM --self_stab=0 --async=0 --run_id=53 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmp62ZXqM/ts.tsv
../cmake-build-debug/iterative_avg_consensu

../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=30 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpos1qn0 --self_stab=0 --async=0 --run_id=65 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpos1qn0/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=30 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpUhgqc4 --self_stab=1 --async=0 --run_id=65 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpUhgqc4/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=30 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpCtFUoO --self_stab=0 --async=0 --run_id=66 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpCtFUoO/ts.tsv
../cmake-build-debug/iterative_avg_consensu

../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=30 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpZgUJmO --self_stab=0 --async=0 --run_id=78 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpZgUJmO/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=30 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpeAM6rh --self_stab=1 --async=0 --run_id=78 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpeAM6rh/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=30 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmp5paV_N --self_stab=0 --async=0 --run_id=79 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmp5paV_N/ts.tsv
../cmake-build-debug/iterative_avg_consensu

../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=30 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpwufjom --self_stab=0 --async=0 --run_id=91 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpwufjom/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=30 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpDkcdgz --self_stab=1 --async=0 --run_id=91 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpDkcdgz/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=30 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpvjNxzE --self_stab=0 --async=0 --run_id=92 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpvjNxzE/ts.tsv
../cmake-build-debug/iterative_avg_consensu

In [17]:
%%time
EPSILON=10e-3
SIM_AVG=1.
meta_args = dict(nnodes=10, distance=40, secs=15, graph_correction=1)
mg = experiment(meta_args, epsilon=EPSILON, sim_avg=SIM_AVG, nbruns=100, xp_name="nnodes_10-distance_40")

../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=40 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpp52CDD --self_stab=0 --async=0 --run_id=0 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpp52CDD/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=40 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmppikPxg --self_stab=1 --async=0 --run_id=0 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmppikPxg/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=40 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmp5imeAY --self_stab=0 --async=0 --run_id=1 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmp5imeAY/ts.tsv
../cmake-build-debug/iterative_avg_consensus -

../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=40 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmp63CLv_ --self_stab=0 --async=0 --run_id=13 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmp63CLv_/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=40 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpZAY2ZL --self_stab=1 --async=0 --run_id=13 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpZAY2ZL/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=40 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmp4gYQdY --self_stab=0 --async=0 --run_id=14 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmp4gYQdY/ts.tsv
../cmake-build-debug/iterative_avg_consensu

../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=40 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpXUP7wd --self_stab=0 --async=0 --run_id=26 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpXUP7wd/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=40 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpbe1HWW --self_stab=1 --async=0 --run_id=26 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpbe1HWW/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=40 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpoYVwN9 --self_stab=0 --async=0 --run_id=27 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpoYVwN9/ts.tsv
../cmake-build-debug/iterative_avg_consensu

../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=40 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmphFfdFH --self_stab=0 --async=0 --run_id=39 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmphFfdFH/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=40 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmp_epCkE --self_stab=1 --async=0 --run_id=39 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmp_epCkE/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=40 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpGb0wTN --self_stab=0 --async=0 --run_id=40 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpGb0wTN/ts.tsv
../cmake-build-debug/iterative_avg_consensu

../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=40 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpvxsdOL --self_stab=0 --async=0 --run_id=52 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpvxsdOL/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=40 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpzM1cZg --self_stab=1 --async=0 --run_id=52 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpzM1cZg/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=40 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpVWaqha --self_stab=0 --async=0 --run_id=53 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpVWaqha/ts.tsv
../cmake-build-debug/iterative_avg_consensu

../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=40 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmp4WcBpH --self_stab=0 --async=0 --run_id=65 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmp4WcBpH/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=40 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpsw3Y4D --self_stab=1 --async=0 --run_id=65 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpsw3Y4D/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=40 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpmmOH0x --self_stab=0 --async=0 --run_id=66 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpmmOH0x/ts.tsv
../cmake-build-debug/iterative_avg_consensu

../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=40 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmp0U7ouP --self_stab=0 --async=0 --run_id=78 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmp0U7ouP/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=40 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpNV7E2h --self_stab=1 --async=0 --run_id=78 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpNV7E2h/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=40 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmp6uaTrG --self_stab=0 --async=0 --run_id=79 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmp6uaTrG/ts.tsv
../cmake-build-debug/iterative_avg_consensu

../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=40 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpCxnZCD --self_stab=0 --async=0 --run_id=91 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpCxnZCD/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=40 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpudZBbK --self_stab=1 --async=0 --run_id=91 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpudZBbK/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=40 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpybsNXT --self_stab=0 --async=0 --run_id=92 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpybsNXT/ts.tsv
../cmake-build-debug/iterative_avg_consensu

In [14]:
%%time
EPSILON=10e-3
SIM_AVG=1.
meta_args = dict(nnodes=10, distance=50, secs=15, graph_correction=1)
mg = experiment(meta_args, epsilon=EPSILON, sim_avg=SIM_AVG, nbruns=100, xp_name="nnodes_10-distance_50")

../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=50 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpB4ztZO --self_stab=0 --async=0 --run_id=0 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpB4ztZO/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=50 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpx4vM_o --self_stab=1 --async=0 --run_id=0 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpx4vM_o/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=50 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmp0iXnyF --self_stab=0 --async=0 --run_id=1 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmp0iXnyF/ts.tsv
../cmake-build-debug/iterative_avg_consensus -

../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=50 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpXf1Xmk --self_stab=0 --async=0 --run_id=13 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpXf1Xmk/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=50 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpW2IQzm --self_stab=1 --async=0 --run_id=13 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpW2IQzm/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=50 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpTZcytq --self_stab=0 --async=0 --run_id=14 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpTZcytq/ts.tsv
../cmake-build-debug/iterative_avg_consensu

../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=50 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpfBAcWJ --self_stab=0 --async=0 --run_id=26 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpfBAcWJ/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=50 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpYRsuDI --self_stab=1 --async=0 --run_id=26 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpYRsuDI/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=50 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpUYZpFf --self_stab=0 --async=0 --run_id=27 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpUYZpFf/ts.tsv
../cmake-build-debug/iterative_avg_consensu

../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=50 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpk4YS4p --self_stab=0 --async=0 --run_id=39 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpk4YS4p/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=50 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpwVdYFR --self_stab=1 --async=0 --run_id=39 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpwVdYFR/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=50 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpO3giWw --self_stab=0 --async=0 --run_id=40 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpO3giWw/ts.tsv
../cmake-build-debug/iterative_avg_consensu

../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=50 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpcJPEkh --self_stab=0 --async=0 --run_id=52 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpcJPEkh/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=50 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpF0lt0F --self_stab=1 --async=0 --run_id=52 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpF0lt0F/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=50 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmp7fvZoE --self_stab=0 --async=0 --run_id=53 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmp7fvZoE/ts.tsv
../cmake-build-debug/iterative_avg_consensu

../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=50 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpISTziO --self_stab=0 --async=0 --run_id=65 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpISTziO/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=50 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmp26OCez --self_stab=1 --async=0 --run_id=65 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmp26OCez/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=50 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmp8NTzPD --self_stab=0 --async=0 --run_id=66 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmp8NTzPD/ts.tsv
../cmake-build-debug/iterative_avg_consensu

../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=50 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmp7gMVyq --self_stab=0 --async=0 --run_id=78 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmp7gMVyq/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=50 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpyMrida --self_stab=1 --async=0 --run_id=78 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpyMrida/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=50 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmp_hQLlA --self_stab=0 --async=0 --run_id=79 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmp_hQLlA/ts.tsv
../cmake-build-debug/iterative_avg_consensu

../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=50 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpQyh4OF --self_stab=0 --async=0 --run_id=91 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpQyh4OF/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=50 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpLLwL2o --self_stab=1 --async=0 --run_id=91 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpLLwL2o/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=50 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpQjbU59 --self_stab=0 --async=0 --run_id=92 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpQjbU59/ts.tsv
../cmake-build-debug/iterative_avg_consensu

In [ ]:
%%time
EPSILON=10e-3
SIM_AVG=1.
meta_args = dict(nnodes=10, distance=60, secs=15, graph_correction=1)
mg = experiment(meta_args, epsilon=EPSILON, sim_avg=SIM_AVG, nbruns=100, xp_name="nnodes_10-distance_60")

../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=60 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpqSRqI6 --self_stab=0 --async=0 --run_id=0 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpqSRqI6/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=60 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpARpvJ8 --self_stab=1 --async=0 --run_id=0 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpARpvJ8/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=60 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpzV1Ymq --self_stab=0 --async=0 --run_id=1 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpzV1Ymq/ts.tsv
../cmake-build-debug/iterative_avg_consensus -

../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=60 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpFluOMC --self_stab=0 --async=0 --run_id=13 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpFluOMC/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=60 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmps5_DMm --self_stab=1 --async=0 --run_id=13 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmps5_DMm/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=60 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpVkjcYT --self_stab=0 --async=0 --run_id=14 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpVkjcYT/ts.tsv
../cmake-build-debug/iterative_avg_consensu

../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=60 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmp4CSXwc --self_stab=0 --async=0 --run_id=26 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmp4CSXwc/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=60 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpgg8rBS --self_stab=1 --async=0 --run_id=26 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpgg8rBS/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=60 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpZ9s4XN --self_stab=0 --async=0 --run_id=27 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpZ9s4XN/ts.tsv
../cmake-build-debug/iterative_avg_consensu

../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=60 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpaBwXA8 --self_stab=0 --async=0 --run_id=39 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpaBwXA8/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=60 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpiNhZmw --self_stab=1 --async=0 --run_id=39 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpiNhZmw/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=60 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpTVtTik --self_stab=0 --async=0 --run_id=40 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpTVtTik/ts.tsv
../cmake-build-debug/iterative_avg_consensu

../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=60 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpj6L3Gv --self_stab=0 --async=0 --run_id=52 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpj6L3Gv/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=60 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpQ2vbL7 --self_stab=1 --async=0 --run_id=52 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpQ2vbL7/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=60 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpc98v0i --self_stab=0 --async=0 --run_id=53 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpc98v0i/ts.tsv
../cmake-build-debug/iterative_avg_consensu

../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=60 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpQtqtFh --self_stab=0 --async=0 --run_id=65 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpQtqtFh/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=60 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmp26flkA --self_stab=1 --async=0 --run_id=65 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmp26flkA/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=60 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpTHozOy --self_stab=0 --async=0 --run_id=66 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpTHozOy/ts.tsv
../cmake-build-debug/iterative_avg_consensu

../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=60 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpXGSj5K --self_stab=0 --async=0 --run_id=78 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpXGSj5K/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=60 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpvnJgeo --self_stab=1 --async=0 --run_id=78 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpvnJgeo/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=60 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpQCMDfz --self_stab=0 --async=0 --run_id=79 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpQCMDfz/ts.tsv
../cmake-build-debug/iterative_avg_consensu

../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=60 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpFwrIK4 --self_stab=0 --async=0 --run_id=91 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpFwrIK4/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=60 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpdRqFpn --self_stab=1 --async=0 --run_id=91 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpdRqFpn/ts.tsv
../cmake-build-debug/iterative_avg_consensus --nNodes=10 --distance=60 --secsToRun=15 --makhoul=0 --path=/var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpzZDD3L --self_stab=0 --async=0 --run_id=92 --graph_correction=1  2>&1 >/dev/null | grep "^TR" > /var/folders/54/w9nqxzfs4h17pdmbb40dzwfm0000gn/T/tmpzZDD3L/ts.tsv
../cmake-build-debug/iterative_avg_consensu

In [3]:
EPSILON=10e-3
SIM_AVG=1.
line_no = 1

with open("scaling_experiments.sh", "w+") as fp:
    for run_id in range(100):
        for nnode in [20, 40, 60, 80]:
            for distance in [10]:
                fp.write("python run_experiment.py ../cmake-build-release/iterative_avg_consensus %d %d %d\n" %(run_id, nnode, distance))
#                meta_args = dict(nnodes=nnode, distance=distance, secs=20, graph_correction=1)
#                mg = experiment(meta_args, epsilon=EPSILON, sim_avg=SIM_AVG, force_run_id=run_id, 
#                                xp_name="nnodes_%d-distance_%d" %(nnode, distance))
                line_no += 1